In [6]:
from lazypredict.Supervised import LazyClassifier

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
data = pd.read_csv('../data/preprocessed/seeds_data.csv', index_col=[0])

In [12]:
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
preprocessor_pipe = Pipeline([
    ('power_transformer', PowerTransformer())
])

X = preprocessor_pipe.fit_transform(data.drop(columns=['target'], axis=1).to_numpy())
Y = data['target'].to_numpy()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=123)

In [14]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

In [15]:
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 30/30 [00:00<00:00, 50.03it/s]

[06:01:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [16]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearSVC,0.97,0.98,None,0.98,0.01
LinearDiscriminantAnalysis,0.97,0.98,None,0.98,0.01
XGBClassifier,0.97,0.98,None,0.98,0.06
SGDClassifier,0.97,0.98,None,0.98,0.01
RidgeClassifierCV,0.97,0.98,None,0.98,0.01
RidgeClassifier,0.97,0.98,None,0.98,0.01
RandomForestClassifier,0.97,0.98,None,0.98,0.11
PassiveAggressiveClassifier,0.97,0.98,None,0.98,0.01
NearestCentroid,0.97,0.98,None,0.98,0.01


In [17]:
from mlxtend.classifier import StackingCVClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier, PassiveAggressiveClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

classifiers = (PassiveAggressiveClassifier(), RidgeClassifier(), CalibratedClassifierCV())

In [18]:
stacked_classifier = StackingCVClassifier(classifiers=classifiers, meta_classifier=RandomForestClassifier())

In [19]:
from sklearn.pipeline import Pipeline
clf_pipe = Pipeline([
    ('power_transformer', PowerTransformer()),
    ('stacked_classifier', stacked_classifier)
])

In [20]:
clf_pipe.fit(X_train, y_train)


Pipeline(steps=[('power_transformer', PowerTransformer()),
                ('stacked_classifier',
                 StackingCVClassifier(classifiers=(PassiveAggressiveClassifier(),
                                                   RidgeClassifier(),
                                                   CalibratedClassifierCV()),
                                      meta_classifier=RandomForestClassifier()))])

In [21]:
clf_pipe.score(X_test, y_test)

0.975

In [22]:
from joblib import dump

In [23]:
dump(clf_pipe, '../models/clf_pipe')

['../models/clf_pipe']

In [27]:
x_prediction_test = np.random.rand(100,7)

In [28]:
clf_pipe.predict(x_prediction_test)

array([2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2,
       1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1,
       2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 3, 2, 1, 2, 2,
       1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2])